In [ ]:
# -----------------------------------------------------------------
# Notebook de Limpieza: Organizations
# -----------------------------------------------------------------
#
# Objetivo: Leer el archivo raw 'organizations_raw.csv', aplicar un
# proceso de limpieza completo y guardar el resultado en la carpeta 'processed'.
#
# Autor: Tu Nombre
# Fecha: 2023-10-27
#
# -----------------------------------------------------------------

# 1. Carga de Datos
En esta sección, cargamos el archivo raw y realizamos una exploración inicial para entender su estado.

Celda 1: Configuración e Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# --- CONFIGURACIÓN ---
# ¡Cambia estas variables para cada nuevo archivo!
RAW_FILE_NAME = 'organizations_raw.csv'
PROCESSED_FILE_NAME = 'organizations.csv'

# Rutas del proyecto (esto no debería cambiar si tu estructura es consistente)
PROJECT_ROOT = Path('.').resolve().parent # Asumiendo que el notebook está en una carpeta 'notebooks'
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'

# Construir las rutas completas
raw_path = RAW_DIR / RAW_FILE_NAME
processed_path = PROCESSED_DIR / PROCESSED_FILE_NAME

Celda 2: Cargar el CSV

In [ ]:
# Cargar el archivo CSV en un DataFrame de Pandas
try:
    df = pd.read_csv(raw_path, delimiter=';')
    print(f"✅ Archivo '{RAW_FILE_NAME}' cargado exitosamente.")
    print(f"El DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
except FileNotFoundError:
    print(f"❌ ERROR: No se pudo encontrar el archivo en la ruta: {raw_path}")

Celda 3: Exploración Inicial


In [ ]:
# Un primer vistazo a los datos
print("--- Primeras 5 filas ---")
display(df.head())

print("\n--- Información del DataFrame y tipos de datos ---")
df.info()

print("\n--- Estadísticas descriptivas de las columnas numéricas ---")
display(df.describe())

print("\n--- Conteo de valores nulos por columna ---")
print(df.isnull().sum())

# 2. Proceso de Limpieza
Aquí aplicamos la checklist de limpieza paso a paso.
### 2.1. Limpieza Estructural
Celda 4: Estandarizar Nombres de Columnas

In [ ]:
print("Nombres de columna originales:", df.columns.tolist())
df.columns = [col.lower().replace(' ', '_').strip() for col in df.columns]
print("Nombres de columna estandarizados:", df.columns.tolist())

Celda 5: Eliminar Duplicados

In [ ]:
initial_rows = len(df)
df.drop_duplicates(inplace=True)
final_rows = len(df)
print(f"Se eliminaron {initial_rows - final_rows} filas duplicadas. Quedan {final_rows} filas.")

### 2.2. Corrección de Tipos de Datos
Celda 6: Convertir columnas a numéricas

In [ ]:
# Lista de columnas que deberían ser numéricas
numeric_cols = ['zip', 'revenue', 'utilization']

for col in numeric_cols:
    # 'coerce' convierte los valores no válidos en NaN (Nulo)
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Tipos de datos después de la conversión a numérico:")
df[numeric_cols].info()

### 2.3. Manejo de Valores Nulos
Celda 7: Imputar (rellenar) valores nulos

In [ ]:
print("Valores nulos ANTES de la imputación:")
print(df.isnull().sum())

# Estrategia de imputación
df['revenue'].fillna(0, inplace=True)
df['utilization'].fillna(0, inplace=True) # Asumimos 0 si no hay dato
df['phone'].fillna('No disponible', inplace=True)
# ... añade otras reglas de imputación aquí ...

# Eliminar filas donde columnas clave son nulas
df.dropna(subset=['id', 'name'], inplace=True)

print("\nValores nulos DESPUÉS de la imputación y borrado:")
print(df.isnull().sum())

### 2.4. Limpieza de Contenido y Formato
Celda 8: Limpiar columnas de texto

In [ ]:
# Quitar espacios en blanco al principio y al final
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].str.strip()

# Estandarizar mayúsculas/minúsculas
df['state'] = df['state'].str.upper()
df['city'] = df['city'].str.title()

print("Ejemplo de la columna 'city' después de la limpieza:")
print(df['city'].head())

### 2.5. Validación de Datos
Celda 9: Filtrar valores fuera de rango o inválidos

In [ ]:
# Ejemplo: Mantener solo filas donde el ID tiene una longitud específica
initial_rows = len(df)
df = df[df['id'].str.len() == 36]
print(f"Se eliminaron {initial_rows - len(df)} filas con ID de longitud inválida.")

# Ejemplo: Asegurar que utilization esté en un rango lógico
df = df[(df['utilization'] >= 0) & (df['utilization'] <= 100)]

# 3. Verificación Final y Guardado
Última revisión del DataFrame limpio antes de guardarlo.

Celda 10: Verificación Final

In [ ]:
print("--- Información final del DataFrame limpio ---")
df.info()

print("\n--- Conteo final de nulos ---")
print(df.isnull().sum())

print("\n--- Muestra final de los datos ---")
display(df.head())

Celda 11: Guardar el archivo procesado

In [ ]:
# Asegurarse de que el directorio de salida exista
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Guardar el DataFrame limpio a un nuevo archivo CSV
df.to_csv(processed_path, index=False, sep=';')

print(f"✅ Proceso completado. Archivo limpio guardado en: \n{processed_path}")